In [1]:
import pandas as pd
import numpy as np

In [3]:
# prepare data
   
ratdata= pd.read_csv("ratdataNormChecked.csv")

ratdata.columns
    

Index(['id', 'day', 'weight', 'gender', 'liverind', 'heartind', 'appind',
       'femstate', 'gonfatind', 'batind', 'sulcer', 'kmethod', 'tailind',
       'blength', 'place', 'year', 'ADWBind', 'gonind', 'BMI'],
      dtype='object')

In [6]:
ratdata.head()

,id,day,weight,gender,liverind,heartind,appind,femstate,gonfatind,batind,sulcer,kmethod,tailind,blength,place,year,ADWBind,gonind,BMI
0,rat1,67,22.0,1,0.036364,0.005091,0.013636,4,0.000000,0.001864,1,3,0.714286,10.5,2,5,0.454545,0.000000,0.199546
1,rat2,251,182.0,1,0.010440,0.004396,0.008791,4,0.023077,0.000742,3,3,0.666667,19.5,1,3,0.302198,1.648659,0.478632
2,rat3,230,37.5,1,0.051200,0.006133,0.020800,4,0.000000,0.001467,2,1,0.904762,10.5,3,8,0.653333,0.000000,0.340136
3,rat4,261,45.0,2,0.047333,0.005556,0.105111,,0.293333,0.002178,1,1,0.869565,11.5,9,2,0.260000,2.653242,0.340265
4,rat5,262,43.0,1,0.052558,0.005349,0.021628,4,0.000000,0.001884,1,1,0.720000,12.5,3,8,0.500000,0.000000,0.275200


The data is presented in a transaction form.

Attributes:
- ratid (not needed in rule discovery, but doesn't harm, helps to
  identify rats if needed; note that each id is now a binary attribute)
- ###summer if day 116...300 and winter if day 1...115 or 301...365
- ###freezer if day=0
- ###weightlow if weight<=162 and weightnormal otherwise (these probably
  correspond to puppies/young and adult rats)
- ###gender  female male
- ###femstate
- ###liversmall if liverind<=0.037 and liverlarge if liverind>0.064
- ###pregnant and nursing from femstate
- ###batlow if batind<=0.00067 and bathigh if batind>0.00184
- ###tailshort if tailind<=0.74 and taillong if tailind>0.85
- ###wild if place=1-3, and lab otherwise
- adrenalsmall if ADWBind<=0.21 and adrenalarge if ADWBind>0.48
- BMIsmall if BMI<=0.47 and BMIlarge if BMI>0.75
feature num=22+9
- Decided by self-----
- 'heartind',   heartsmall  <0.0036   heartlarge >0.0046 
- 'appind',     appsmall  <0.0104    applarge >0.0167
- 'gonfatind',   gonsmall <0.0018    gonfat >0.0103
- ####'sulcer',      sulcermild <=4      sulcerserious >4
- ####'kmethod',    1-5  one binary attribute for each place
- 'blength',    blengthshort <19.5   blengthlong>22.5
- 'year',       


Description of features 

* id - rat identifier
* day - day of the year, normal days 1-366 (from 1st Jan), and special code
  0=freezer (date not known). 
* weight - body weight (g). This can be used also to detect puppies (very small rats);
  adults weigh normally 140-500g.
* gender - 1=female, 2=male
* adrenal - adrenal gland weight (mg), important marker for stress, see ADWBWind
* liver - liver weight (g)
* heart - heart weight (g)
* appendix - appendix weight (g)
* femstate - if the rat was pregnant (1), nursing (2), pregrant and nursing (3), or
  neither (4). Missing for male rats. 
* gonfat - weight of gonadal fat (g); tells on food intake (more fat) and may reflect
  stress (less fat).
* bat - weight of another type of fat, brown adipose tissue (g). This is healthy fat.
  Young animals have more of it and also cold exposure increases its amount.
* sulcer - evaluation of stomach ulcer (gastric ulcer), 1, 2, ..., 8, where
  1=none, 2=mild, and 8=very serious. 
* kmethod killing method, codes 1-5, meaning not known (a few trap types, shooting, gas)
* tail - tail length (cm); adults have typically 10.5-24cm long tail. It
  is hypothesized that a long tail would be beneficial for the rat. 
* blength - body length (cm); adults typically 15-28cm.
* place - catching place, number codes 1-9, 1-3 are wild rats from 3 places
  (1 and 3 are landfills, 2 a fur farm) and 4-9 laboratory rats in different
  experiments (7 and 8 got extra melatonin). There can be differences between
  wild vs. laboratory rats or between wild rats from different places.
* intestine - length of intestine 
* year - code for the year (last digit on the same decade). Missing if unknown
  (see day).
* ADWBWind adrenal gland weight normalized by the body weight; an important
  stress indicator (chronic stress increases adrenal glands)
* gonind - gonad fat index. Derived from gonad fat and some other features,
  but the equation unknown. Can be an important welfare index.

In [8]:
ratdata.describe()

,day,weight,gender,liverind,heartind,appind,gonfatind,batind,sulcer,kmethod,tailind,blength,place,ADWBind,gonind,BMI
count,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000
mean,161.272727,266.409091,1.372549,0.052679,0.004178,0.014237,0.009235,0.001389,1.433155,1.670232,0.783175,20.460071,3.402852,0.387096,0.953921,0.604386
std,87.565703,99.339288,0.483915,0.013877,0.000958,0.006435,0.017236,0.000616,0.846994,1.036816,0.060452,2.901651,2.093764,0.131196,0.769858,0.129435
min,0.000000,22.000000,1.000000,0.010440,0.002215,0.002949,0.000000,0.000031,1.000000,1.000000,0.500000,10.500000,1.000000,0.052299,0.000000,0.199546
25%,90.000000,204.000000,1.000000,0.042769,0.003614,0.010417,0.001838,0.000930,1.000000,1.000000,0.750000,19.500000,3.000000,0.292632,0.262364,0.514873
50%,169.000000,276.000000,1.000000,0.051402,0.004098,0.013456,0.005686,0.001287,1.000000,1.000000,0.778846,21.000000,3.000000,0.380252,0.970779,0.608889
75%,244.000000,339.000000,2.000000,0.061373,0.004630,0.016777,0.010379,0.001720,2.000000,3.000000,0.824242,22.500000,3.000000,0.466942,1.410987,0.695000
max,354.000000,500.000000,2.000000,0.105678,0.015326,0.105111,0.293333,0.004866,5.000000,4.000000,0.986667,26.500000,9.000000,0.974359,3.945264,1.005949


In [87]:
# Conversion of features
Binary_ratdata=ratdata.copy()

#num conversion

Binary_ratdata.weight=pd.cut(ratdata.weight, bins=[0,162,500],labels=["weightlow", "weightnormal" ])

Binary_ratdata["day"]=Binary_ratdata["day"].astype("int")
Binary_ratdata["day"]=Binary_ratdata["day"]%300
Binary_ratdata["day"]=pd.cut(ratdata.day, bins=[0,0.5,115,300],labels=["freezer","winter", "summer" ])

Binary_ratdata["sulcer"]=pd.cut(ratdata.sulcer, bins=[0,4,8],labels=["sulcermild","sulcerserious"])

Binary_ratdata["place"]=pd.cut(ratdata.place, bins=[0,3,8],labels=["wild","lab"])

Binary_ratdata["liverind"]=pd.cut(ratdata.liverind, bins=[0,0.037,0.064,1],labels=["liversmall","livernormal","liverlarge"])

Binary_ratdata["batind"]=pd.cut(ratdata.batind, bins=[0,0.00067,0.00184,1],labels=["batsmall","batnormal","batlarge"])

Binary_ratdata["tailind"]=pd.cut(ratdata.tailind, bins=[0,0.74,0.85,1],labels=["tailsmall","tailnormal","taillarge"])

Binary_ratdata["ADWBind"]=pd.cut(ratdata.ADWBind, bins=[0,0.21,0.48,1],labels=["adrenalsmall","adrenalnormal","adrenalarge"])

Binary_ratdata["BMI"]=pd.cut(ratdata.BMI, bins=[0,0.47,0.75,1],labels=["BMIsmall","BMInormal","BMIlarge"])

Binary_ratdata['heartind']=pd.cut(ratdata.heartind, bins=[0,0.0036,0.0046,1],labels=["heartsmall","heartnormal","heartlarge"])

Binary_ratdata['appind']=pd.cut(ratdata.appind, bins=[0,0.0104,0.0167,1],labels=["appsmall","appnormal","applarge"])

Binary_ratdata['gonfatind']=pd.cut(ratdata.gonfatind, bins=[0,0.0018,0.0103,1],labels=["gonsmall","gonnormal","gonfat"])

Binary_ratdata['blength']=pd.cut(ratdata.blength, bins=[0,19.5,22.5,30],labels=["blengthshort","blengthnormal","blengthlong"])

# cat conversion  femstate kmethod year

Binary_ratdata.gender=pd.cut(ratdata.gender, bins=[0,1,2],labels=["female", "male" ])

Binary_ratdata.loc[Binary_ratdata["femstate"]=="1",["femstate"]]="pregnant"
Binary_ratdata.loc[Binary_ratdata["femstate"]=="2",["femstate"]]="nursing"
Binary_ratdata.loc[Binary_ratdata["femstate"]=="3",["femstate"]]="pregnant+nursing"
Binary_ratdata.loc[Binary_ratdata["femstate"]=="4",["femstate"]]="femstate_neither"

Binary_ratdata.loc[Binary_ratdata["kmethod"]==1,["kmethod"]]="kmethod1"
Binary_ratdata.loc[Binary_ratdata["kmethod"]==2,["kmethod"]]="kmethod2"
Binary_ratdata.loc[Binary_ratdata["kmethod"]==3,["kmethod"]]="kmethod3"
Binary_ratdata.loc[Binary_ratdata["kmethod"]==4,["kmethod"]]="kmethod4"
Binary_ratdata.loc[Binary_ratdata["kmethod"]==5,["kmethod"]]="kmethod5"


Binary_ratdata.head()

,id,day,weight,gender,liverind,heartind,appind,femstate,gonfatind,batind,sulcer,kmethod,tailind,blength,place,year,ADWBind,gonind,BMI
0,rat1,winter,weightlow,female,liversmall,heartlarge,appnormal,femstate_neither,NaN,batlarge,sulcermild,kmethod3,tailsmall,blengthshort,wild,5,adrenalnormal,0.000000,BMIsmall
1,rat2,summer,weightnormal,female,liversmall,heartnormal,appsmall,femstate_neither,gonfat,batnormal,sulcermild,kmethod3,tailsmall,blengthshort,wild,3,adrenalnormal,1.648659,BMInormal
2,rat3,summer,weightlow,female,livernormal,heartlarge,applarge,femstate_neither,NaN,batnormal,sulcermild,kmethod1,taillarge,blengthshort,wild,8,adrenalarge,0.000000,BMIsmall
3,rat4,summer,weightlow,male,livernormal,heartlarge,applarge,,gonfat,batlarge,sulcermild,kmethod1,taillarge,blengthshort,NaN,2,adrenalnormal,2.653242,BMIsmall
4,rat5,summer,weightlow,female,livernormal,heartlarge,applarge,femstate_neither,NaN,batlarge,sulcermild,kmethod1,tailsmall,blengthshort,wild,8,adrenalarge,0.000000,BMIsmall


In [92]:
# original_cols=['id', 'day', 'weight', 'gender', 'liverind', 'heartind', 'appind',
#        'femstate', 'gonfatind', 'batind', 'sulcer', 'kmethod', 'tailind',
#        'blength', 'place', 'year', 'ADWBind', 'gonind', 'BMI']

keep_columns=['id', 'day', 'weight', 'gender', 'liverind', 'heartind', 'appind',
       'femstate', 'gonfatind', 'batind', 'sulcer', 'kmethod', 'tailind',
       'blength', 'place',  'ADWBind', 'BMI']

save_data=Binary_ratdata[keep_columns]

In [94]:
save_data.to_csv("ratdata.txt",header=None, index=None, sep=' ', mode='a')